In [1]:
import numpy as np
import ta
import matplotlib.pyplot as plt
from utils.ticker_data import get_stock_data

In [32]:
df = get_stock_data("^GSPC", start="2025-07-01", hourly=True)
df.head()

c:\Users\borishan\Desktop\ml\code\ml_vs_baseline_reits\utils\ticker_data.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Volume,logReturns
0,2025-07-01 14:30:00+00:00,6193.609863,6199.000000,6178.209961,6195.879883,570669686,0.000347
1,2025-07-01 15:30:00+00:00,6195.759766,6203.549805,6183.529785,6196.140137,442759000,0.000042
2,2025-07-01 16:30:00+00:00,6196.089844,6205.290039,6193.169922,6203.490234,342618000,0.001186
3,2025-07-01 17:30:00+00:00,6204.390137,6210.779785,6200.790039,6202.859863,272564000,-0.000102
4,2025-07-01 18:30:00+00:00,6202.859863,6207.589844,6190.310059,6206.120117,328349000,0.000525


In [38]:
df['Datetime'].dt.hour

0     14
1     15
2     16
3     17
4     18
5     19
6     20
7     13
8     14
9     15
10    16
11    17
12    18
13    19
14    20
15    13
16    14
17    15
18    17
19    13
20    14
21    15
22    16
23    17
24    18
25    19
26    20
27    13
28    14
29    15
30    16
31    17
32    18
33    19
34    20
35    13
36    14
37    15
38    16
39    17
40    18
41    19
42    20
43    21
44    13
45    14
46    15
47    16
48    17
49    18
50    19
51    20
52    13
53    14
54    15
55    16
56    17
57    18
58    19
59    20
Name: Datetime, dtype: int32

In [27]:
def prepare_data(df, horizon=1):
    if 'Datetime' not in df:
        return ValueError('need hourly data')
    
    df = df.set_index('Datetime')
    
    df['Return'] = df['Close'].pct_change()
    df['Volatility_24h'] = df['Return'].rolling(24).std()
    
    df['RSI_14'] = ta.momentum.rsi(df['Close'], window=14)
    df['MACD'] = ta.trend.macd_diff(df['Close'])
    df['BB_upper'], df['BB_middle'], df['BB_lower'] = ta.volatility.bollinger_hband(df['Close']), ta.volatility.bollinger_mavg(df['Close']), ta.volatility.bollinger_lband(df['Close'])
    df['ATR_14'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], window=14)
    
    df['DayOfWeek'] = df.index.dayofweek
    df['Hour'] = df.index.hour
    df['Hour_sin'] = np.sin(2 * np.pi * df['Hour']/24)
    df['Hour_cos'] = np.cos(2 * np.pi * df['Hour']/24)
    
    for lag in [1, 4, 12, 24, 48]:
        df[f'Return_lag_{lag}'] = df['Return'].shift(lag)
        df[f'Volume_lag_{lag}'] = df['Volume'].shift(lag)
    
    df['Target'] = df['Close'].shift(-horizon)
    
    df = df.dropna()
    
    features = df.drop(columns=['Target'])
    target = df['Target']
    
    return features, target

In [29]:
X, y = prepare_data(df)
(X.shape, y.shape)

((10, 28), (10,))

In [31]:
X.head()

,Open,High,Low,Close,Volume,logReturns,Return,Volatility_24h,RSI_14,MACD,...,Return_lag_1,Volume_lag_1,Return_lag_4,Volume_lag_4,Return_lag_12,Volume_lag_12,Return_lag_24,Volume_lag_24,Return_lag_48,Volume_lag_48
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-07-10 18:30:00+00:00,6287.669922,6288.100098,6282.680176,6283.490234,278196000,-0.000667,-0.000666,0.001527,67.593361,3.041695,...,0.000809,237423000.0,0.002862,361940471.0,-0.000407,247022700.0,0.002152,409952000.0,0.000042,442759000.0
2025-07-10 19:30:00+00:00,6283.439941,6287.850098,6274.089844,6280.140137,431523000,-0.000533,-0.000533,0.001535,64.980334,2.317822,...,-0.000666,278196000.0,0.000064,268395526.0,0.000796,212353000.0,-0.000125,0.0,0.001186,342618000.0
2025-07-10 20:00:00+00:00,6280.250000,6280.460000,6280.190000,6280.460000,0,0.000051,0.000051,0.001501,65.118984,1.657756,...,-0.000533,431523000.0,0.000883,217946000.0,0.000845,244781000.0,-0.001220,540725372.0,-0.000102,272564000.0
2025-07-11 13:30:00+00:00,6255.680176,6264.209961,6237.600098,6261.970215,429044324,-0.002948,-0.002944,0.001646,52.242800,-0.131881,...,0.000051,0.0,0.000809,237423000.0,0.001137,259119000.0,-0.000100,433470649.0,0.000526,328349000.0
2025-07-11 14:30:00+00:00,6262.000000,6265.450195,6249.180176,6254.660156,335952796,-0.001168,-0.001167,0.001557,48.186082,-1.821932,...,-0.002944,429044324.0,-0.000666,278196000.0,0.000767,346840000.0,0.003076,261095979.0,-0.001376,472896000.0
